In [1]:
!pip install pytorch-metric-learning
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Sat Feb  3 04:42:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import numpy as np
import pandas as pd
from pytorch_metric_learning import miners, losses, distances
import os
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pickle
import time
from sklearn.model_selection import train_test_split
import json
from tqdm.auto import tqdm
import random
import math
import faiss

In [4]:
train = pd.read_csv("/content/drive/MyDrive/qa_dacon/train.csv")
test = pd.read_csv("/content/drive/MyDrive/qa_dacon/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/qa_dacon/sample_submission.csv")

In [5]:
train.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [6]:
test.head()

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?


In [ ]:
#model_name = "vaiv/kobigbird-roberta-large"
model_name = "monologg/kobigbird-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
device = torch.device("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
class GetData(Dataset):
  def __init__(self, df: pd.DataFrame):
    super(GetData, self).__init__()
    self.data = df
  def __getitem__(self, index):
    return self.data.iloc[index]
  def __len__(self):
    return self.data.shape[0]

In [ ]:
class custom_collate(object):
  def __init__(self):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.chunk_size = 512
  def __call__(self, batch):
    q_input_ids, q_attn_masks, q_labels = [], [], []
    a_input_ids, a_attn_masks, a_labels = [], [], []
    ids = 0
    for idx, row in enumerate(batch):
      q1, q2, a1, a2, a3, a4, a5 = row[0], row[1], row[2], row[3], row[4], row[5], row[6]
      encoded_q1 = self.tokenizer(q1, max_length=self.chunk_size, padding="max_length", truncation=True, return_tensors="pt")
      encoded_q2 = self.tokenizer(q2, max_length=self.chunk_size, padding="max_length", truncation=True, return_tensors="pt")
      encoded_a1 = self.tokenizer(a1, max_length=self.chunk_size, padding="max_length", truncation=True, return_tensors="pt")
      encoded_a2 = self.tokenizer(a2, max_length=self.chunk_size, padding="max_length", truncation=True, return_tensors="pt")
      encoded_a3 = self.tokenizer(a3, max_length=self.chunk_size, padding="max_length", truncation=True, return_tensors="pt")
      encoded_a4 = self.tokenizer(a4, max_length=self.chunk_size, padding="max_length", truncation=True, return_tensors="pt")
      encoded_a5 = self.tokenizer(a5, max_length=self.chunk_size, padding="max_length", truncation=True, return_tensors="pt")

      q_input_ids.append(encoded_q1["input_ids"])
      q_attn_masks.append(encoded_q1["attention_mask"])
      q_labels.append(ids)

      q_input_ids.append(encoded_q2["input_ids"])
      q_attn_masks.append(encoded_q2["attention_mask"])
      q_labels.append(ids)

      a_input_ids.append(encoded_a1["input_ids"])
      a_attn_masks.append(encoded_a1["attention_mask"])
      a_labels.append(ids)

      a_input_ids.append(encoded_a2["input_ids"])
      a_attn_masks.append(encoded_a2["attention_mask"])
      a_labels.append(ids)

      a_input_ids.append(encoded_a3["input_ids"])
      a_attn_masks.append(encoded_a3["attention_mask"])
      a_labels.append(ids)

      a_input_ids.append(encoded_a4["input_ids"])
      a_attn_masks.append(encoded_a4["attention_mask"])
      a_labels.append(ids)

      a_input_ids.append(encoded_a5["input_ids"])
      a_attn_masks.append(encoded_a5["attention_mask"])
      a_labels.append(ids)

      ids += 1
    q_input_ids = torch.cat(q_input_ids, dim=0)
    q_attn_masks = torch.cat(q_attn_masks, dim=0)
    q_labels = torch.tensor(q_labels)

    a_input_ids = torch.cat(a_input_ids, dim=0)
    a_attn_masks = torch.cat(a_attn_masks, dim=0)
    a_labels = torch.tensor(a_labels)

    return q_input_ids, q_attn_masks, q_labels, a_input_ids, a_attn_masks, a_labels

In [ ]:
train_data, val_data = train_test_split(train, test_size=0.1, random_state=42)

In [ ]:
train_set = GetData(train_data)
val_set = GetData(val_data)

train_dataloader = DataLoader(train_set, batch_size=4, shuffle=True, collate_fn=custom_collate())
val_dataloader = DataLoader(val_set, batch_size=4, shuffle=False, collate_fn=custom_collate())

In [10]:
class MeanPooling(nn.Module):
  def __init__(self):
    super(MeanPooling, self).__init__()
  def forward(self, last_hidden_state, attention_masks):
    input_mask_expanded = attention_masks.unsqueeze(-1).expand(last_hidden_state.size()).float()
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)
    return sum_embeddings / sum_mask

class BERTEmbedder(nn.Module):
  def __init__(self, plm="monologg/kobigbird-bert-base"):
    super(BERTEmbedder, self).__init__()
    self.encoder = AutoModel.from_pretrained(plm)
    self.pool = MeanPooling()
    self.fc = nn.Linear(768, 512)
  def forward(self, input_ids, attention_mask):
    x = self.encoder(input_ids, attention_mask)[0]
    x = self.pool(x, attention_mask)
    x = self.fc(x)
    return x

In [ ]:
best_val_loss = np.inf

encoder = BERTEmbedder()
encoder.to(device)

epochs = 20
params = list(encoder.parameters())
optimizer = torch.optim.AdamW(params, lr=2e-5, eps=1e-8)
t_total = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=t_total)

miner = miners.MultiSimilarityMiner()
loss_func = losses.MultiSimilarityLoss()

encoder.zero_grad()
train_losses, val_losses = [], []

for epoch in tqdm(range(epochs), desc="Epochs", position=0, leave=True, total=epochs):
  train_loss = 0
  encoder.train()
  with tqdm(train_dataloader, unit="batch") as tepoch:
    for step, batch in enumerate(tepoch):
      batch = tuple(t.to(device) for t in batch)
      q_input_ids, q_attn_masks, q_labels, a_input_ids, a_attn_masks, a_labels = batch
      q_embeddings = encoder(q_input_ids, q_attn_masks)
      a_embeddings = encoder(a_input_ids, a_attn_masks)
      full_embeddings = torch.cat((q_embeddings, a_embeddings), dim=0)
      full_labels = torch.cat((q_labels, a_labels))
      hard_pairs = miner(full_embeddings, full_labels)
      loss = loss_func(full_embeddings, full_labels, hard_pairs)
      train_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(params, 1.0)
      optimizer.step()
      scheduler.step()
      encoder.zero_grad()
      tepoch.set_postfix(loss=loss.item())
      time.sleep(0.05)

  val_loss = 0
  encoder.eval()
  for step, batch in tqdm(enumerate(val_dataloader), desc="Validation", unit="batch", total=len(val_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    q_input_ids, q_attn_masks, q_labels, a_input_ids, a_attn_masks, a_labels = batch
    with torch.no_grad():
      q_embeddings = encoder(q_input_ids, q_attn_masks)
      a_embeddings = encoder(a_input_ids, a_attn_masks)
      full_embeddings = torch.cat((q_embeddings, a_embeddings), dim=0)
      full_labels = torch.cat((q_labels, a_labels))
      loss = loss_func(full_embeddings, full_labels)
      val_loss += loss.item()

  avg_train_loss = train_loss / len(train_dataloader)
  avg_val_loss = val_loss / len(val_dataloader)
  print(f"Epoch: {epoch+1} | avg train loss: {avg_train_loss} | avg val loss: {avg_val_loss}")
  train_losses.append(avg_train_loss)
  val_losses.append(avg_val_loss)

  if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss
    torch.save(encoder.state_dict(), "/content/drive/MyDrive/best_model.pt")

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/145 [00:00<?, ?batch/s]

Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Validation:   0%|          | 0/17 [00:00<?, ?batch/s]

Epoch: 1 | avg train loss: 0.827507093446008 | avg val loss: 0.9443001010838676


...

Validation:   0%|          | 0/17 [00:00<?, ?batch/s]

Epoch: 19 | avg train loss: 0.4479525770735124 | avg val loss: 0.906761488493751


  0%|          | 0/145 [00:00<?, ?batch/s]

Validation:   0%|          | 0/17 [00:00<?, ?batch/s]

Epoch: 20 | avg train loss: 0.4167389215837265 | avg val loss: 0.9060548508868498


## Inference

In [7]:
#model_name = "vaiv/kobigbird-roberta-large"
model_name = "monologg/kobigbird-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
device = torch.device("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

In [12]:
checkpoint = torch.load("/content/drive/MyDrive/best_model.pt")
encoder = BERTEmbedder().to(device)
print(encoder.load_state_dict(checkpoint))
encoder.eval()
print()

<All keys matched successfully>



In [18]:
questions = test["질문"].values

all_embeddings = []

for i in tqdm(range(len(questions)), desc="Inference", total=len(questions)):
  question = questions[i]
  encoded_input = tokenizer(question, max_length=512, padding="max_length", truncation=True, return_tensors="pt").to(device)
  input_ids = encoded_input["input_ids"]
  attn_mask = encoded_input["attention_mask"]
  with torch.no_grad():
    embeddings = encoder(input_ids, attn_mask)
  embeddings = embeddings.cpu().numpy()
  all_embeddings.append(embeddings)

Inference:   0%|          | 0/130 [00:00<?, ?it/s]

In [20]:
all_embeddings = np.array(all_embeddings).reshape((-1, 512))

all_embeddings.shape

(130, 512)

In [22]:
submission.iloc[:, 1:] = all_embeddings

In [23]:
submission

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.227893,-0.218297,-0.004735,-0.300609,0.150381,-0.064813,-0.203459,0.027290,-0.278298,...,0.193469,0.029015,-0.154326,0.211614,0.109942,-0.539421,-0.110417,-0.203191,0.350708,0.308456
1,TEST_001,0.461920,-0.076569,0.078509,-0.139696,0.198658,-0.122555,-0.057765,0.087112,-0.004416,...,-0.061665,0.288111,0.108696,-0.192877,-0.315570,-0.226328,0.109583,0.323673,-0.191602,-0.289655
2,TEST_002,-0.258973,-0.297877,-0.214377,0.021932,0.084065,-0.144298,-0.215539,0.000068,-0.004760,...,-0.145309,0.292995,-0.070343,0.289001,0.358458,-0.571643,0.136430,-0.109925,0.354135,0.201094
3,TEST_003,-0.094086,-0.158001,-0.218658,-0.134700,-0.182298,-0.240593,0.101340,-0.255272,-0.841950,...,0.066789,0.000734,0.009258,0.070455,-0.100511,-0.106320,-0.461967,-0.292533,0.024163,-0.097466
4,TEST_004,-0.021253,0.057039,0.220874,-0.050161,-0.390427,0.048453,0.130003,-0.061873,-0.050280,...,0.191527,0.350771,-0.151450,-0.146616,-0.503833,-0.373203,0.119858,0.368239,0.117299,-0.536372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.218314,0.000620,0.447364,0.038151,0.189933,-0.088263,0.084425,0.559141,-0.447097,...,-0.128118,-0.305727,-0.029931,0.046573,-0.167291,-0.202695,-0.180530,-0.011993,0.035601,0.025168
126,TEST_126,-0.144195,-0.476246,-0.189124,0.162938,0.189823,-0.169727,-0.100182,0.038002,-0.406215,...,0.043599,0.329470,0.022997,0.035499,0.072140,-0.293963,-0.134911,0.100183,0.458705,-0.023834
127,TEST_127,0.213073,-0.202469,0.137761,0.257000,-0.184122,-0.082706,0.002658,-0.333635,-0.536855,...,0.104707,0.050945,-0.428539,0.111198,-0.377551,-0.159288,-0.115942,0.218726,-0.176368,0.114784
128,TEST_128,0.038113,-0.180456,-0.265867,0.210824,-0.069154,-0.234425,0.349345,0.064101,-0.082072,...,0.114794,-0.263890,0.273557,0.014790,-0.045276,-0.534854,0.143698,0.131559,-0.063091,-0.588356


In [24]:
submission.to_csv("monoencoder_baseline.csv", index=False)